In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
df = pd.read_csv('GFM_data.csv', sep = '\t')
df.head()
 
#%tensorflow_version 2.x
import tensorflow as tf
import string
import requests
 
text = df.get('Text').to_numpy()
 
def clean_data(txt_elmt):
    # txt_elmt is each element (each post) in the "text" array
    tokens = txt_elmt.split() # each word

    # nothing changes. except removing punctuation
    mytable = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(mytable) for word in tokens]
    return [word.lower() for word in tokens if word.isalpha()]
 
#for i in text:
# try with text[0] first
tokens = clean_data(text[0]) # list of tokens
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
sufficiency = 200_000
for i in range(len(tokens)):
    sequence = tokens[i: i + length] # every 50-word
    lines += [' '. join(sequence)]
    if i > sufficiency:
        break

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(lines)
sequences = np.array(tokenizer.texts_to_sequences(lines), dtype=object) # all words become numbers
 
##X, y = sequences[:, :-1], sequences[:, -1] 
 
# training and test sets
X = []
y = []
for i in sequences:
    X += [i[:-1]]
    y += [i[-1]]
 
 
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
sequence_length = len(X[0])
 
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax')) # we predict each word ==> use size = vocab_size, activation = softmax to get the prob for each predicted word
model.summary()
model.compile(loss='CosineSimilarity', optimizer='sgd', metrics=['accuracy'])
X = np.array(X)
y = np.array(y)
model.fit(X, y, batch_size=256, epochs=100)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 50)            6100      
_________________________________________________________________
lstm_10 (LSTM)               (None, 50, 100)           60400     
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 122)               12322     
Total params: 169,322
Trainable params: 169,322
Non-trainable params: 0
_________________________________________________________________


<ipython-input-75-d5cc8798ff6f>:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(X)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [6]:
import numpy
import sys
import nltk
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pandas as pd

def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

df = pd.read_csv('GFM_data.csv', sep = '\t')
df = df.dropna()
text = df.get('Text')
processed_inputs = text.apply(tokenize_words)
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)
seq_length = 100
x_data = []
y_data = []
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gauri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total number of characters: 836
Total vocab: 833


KeyError: 133

In [76]:
import torch
torch.manual_seed(0)
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt

def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertModel.from_pretrained('bert-base-uncased')

df = df.dropna()
sentences = df.get('Text').apply(tokenize_words)
input_ids = []
attention_masks = []
tokenized_texts = []
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True,
                        truncation=True,
                        max_length = 214,
                        pad_to_max_length = 'longest',                        
                        return_tensors = 'pt',    
                   )
    # Save tokens from sentence as a separate array. 
    marked_text = "[CLS] " + sent + " [SEP]"
    tokenized_texts.append(tokenizer.tokenize(marked_text))
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])

# Convert the list into tensor.
input_ids = torch.cat(input_ids, dim = 0)
segments_ids = torch.ones_like(input_ids)
with torch.no_grad():
    outputs = model(input_ids, segments_ids, output_hidden_states = True)
    hidden_states = outputs.hidden_states
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
print ("Number of batches:", len(hidden_states[0]))
print ("Number of tokens:", len(hidden_states[0][0]))
print ("Number of hidden units:", len(hidden_states[0][0][0]))

C:\Users\Gauri\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2137: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 836
Number of tokens: 214
Number of hidden units: 768


In [80]:
# Concatenate the tensors for all layers. 
token_embeddings = torch.stack(hidden_states, dim=0)
# Swap dimensions, so we get tensors in format: [sentence, tokens, hidden layes, features]
token_embeddings = token_embeddings.permute(1,2,0,3)
processed_embeddings = token_embeddings[:, :, 9:, :]
embeddings = torch.reshape(processed_embeddings, (4, 214, -1))
for i, token_str in enumerate(tokenized_texts[0]):
    print(i, token_str)

0 [CLS]
1 august
2 23rd
3 son
4 shot
5 multiple
6 times
7 back
8 ken
9 ##osh
10 ##a
11 police
12 department
13 officer
14 son
15 broke
16 alter
17 ##cation
18 unrelated
19 party
20 shooting
21 left
22 son
23 critically
24 injured
25 fights
26 life
27 extent
28 son
29 injuries
30 unknown
31 remain
32 prayer
33 ##ful
34 continues
35 undergo
36 multiple
37 rounds
38 operations
39 save
40 life
41 fund
42 established
43 cover
44 son
45 medical
46 expenses
47 mental
48 grief
49 counseling
50 family
51 assist
52 family
53 days
54 come
55 continue
56 seek
57 justice
58 jacob
59 portion
60 proceeds
61 also
62 used
63 benefit
64 son
65 six
66 children
67 witnessed
68 horrific
69 act
70 violence
71 anyone
72 wishing
73 send
74 cards
75 letters
76 encouragement
77 contributions
78 form
79 money
80 order
81 check
82 may
83 mail
84 julia
85 jackson
86 c
87 ben
88 cr
89 ##ump
90 law
91 pl
92 ##lc
93 122
94 calhoun
95 street
96 tall
97 ##aha
98 ##ssee
99 fl
100 323
101 ##01
102 at
103 ##t
104 ##n
105 

In [81]:
from scipy.spatial.distance import cosine
 
sons = cosine(embeddings[0][55], embeddings[0][88])
police_dept = cosine(embeddings[0][18], embeddings[0][19])
police_son = cosine(embeddings[0][5], embeddings[0][18])

print('Distance for two sons:  %.2f' % police)
print('Distance from police to department:  %.2f' % police_dept)
print('Distance from police to son:  %.2f' % police_son)


Distance for two sons:  0.44
Distance from police to department:  0.68
Distance from police to son:  0.73


In [94]:
X, y = input_ids, outputs.last_hidden_state
seq_length = X.shape[1]
vocab_size = tokenizer.vocab_size

In [95]:

modelLTSM = tf.keras.Sequential()
modelLTSM.add(Embedding(vocab_size, 50, input_length=seq_length))
modelLTSM.add(LSTM(100, return_sequences=True))
modelLTSM.add(LSTM(100))
modelLTSM.add(Dense(100, activation='relu'))
modelLTSM.add(Dense(vocab_size, activation='softmax'))
print(modelLTSM.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 214, 50)           1526100   
_________________________________________________________________
lstm_16 (LSTM)               (None, 214, 100)          60400     
_________________________________________________________________
lstm_17 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 30522)             3082722   
Total params: 4,759,722
Trainable params: 4,759,722
Non-trainable params: 0
_________________________________________________________________
None


In [96]:
# compile model
modelLTSM.compile(loss='CosineSimilarity', optimizer='sgd', metrics=['accuracy'])
# fit model
#X = np.array(X)
#y = np.array(y)
modelLTSM.fit(X, y, batch_size=128, epochs=100)

TypeError: Expected DataType for argument 'Tout' not torch.int64.

In [79]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

# clean document
tokens = df.get('Text').apply(clean_doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(tokens.unique()))

0      [on, august, my, son, was, shot, multiple, tim...
1      [the, navajo, nation, fund, has, been, establi...
2      [on, sunday, april, sylvia, leroy, a, pregnant...
3      [first, thank, you, for, being, here, tommy, r...
4      [my, name, is, melissa, green, and, i, am, the...
                             ...                        
208    [the, micho, family, experienced, a, devastati...
209    [my, name, is, faith, roussel, and, i, am, rai...
210    [this, go, fund, me, is, to, put, together, fu...
211    [support, glass, fire, survivors, caryn, fried...
212    [waheguru, ji, ka, khalsa, waheguru, ji, ki, f...
Name: Text, Length: 200, dtype: object
Total Tokens: 836


TypeError: unhashable type: 'list'

In [ ]:

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

In [ ]:

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

In [ ]:


import string
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# clean document
tokens = df.get('Text').apply(clean_doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)


# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')